# Task 1: Exploratory Data Analysis of Brent Oil Prices
**Objective**: Analyze time series properties (trend, stationarity, volatility) to inform Bayesian change point modeling decisions.

**Data Source**: Historical Brent oil prices (May 1987 – Nov 2022), daily closing prices in USD/barrel

**Key Questions**:
1. Does the series exhibit a deterministic trend requiring detrending?
2. Is the series stationary? (Critical for change point modeling)
3. How does volatility cluster around major events?
4. What transformations (e.g., log returns) improve modeling suitability?


In [1]:
# Setup and Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf


# Configure visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

In [4]:
from src.data_loader import load_brent_data
from src.time_series_analysis import adf_test, plot_price_trend

df = load_brent_data('data/brent_daily.csv')
print(adf_test(df['Price'], "Raw Prices"))
plot_price_trend(df)

ModuleNotFoundError: No module named 'src'

In [6]:
# Setup and Imports
import pandas as pd
import numpy as np

# 1. DATA LOADING & VALIDATION
def load_brent_data(filepath: str = "../data/raw/BrentOilPrices.csv") -> pd.DataFrame:
    """
    Load Brent oil price data with robust datetime parsing and validation.
    
    Returns:
        pd.DataFrame with 'Date' (datetime) and 'Price' (float) columns
    """
    try:
        # Parse dates flexibly to handle mixed formats in raw data
        df = pd.read_csv(
            filepath, 
            parse_dates=['Date'],
            date_parser=lambda x: pd.to_datetime(x, format='mixed', dayfirst=True)
        )
        
        # Standardize column names and sort chronologically
        df.columns = ['Date', 'Price']
        df = df.sort_values('Date').reset_index(drop=True)
        
        # Validate data continuity
        _validate_date_gaps(df)
        
        print(f"✓ Loaded {len(df):,} daily observations from {df['Date'].min().date()} to {df['Date'].max().date()}")
        print(f"✓ Price range: ${df['Price'].min():.2f} – ${df['Price'].max():.2f}")
        return df
    except FileNotFoundError:
        raise FileNotFoundError(f"Data file not found at {filepath}. Ensure path is correct.")
    except Exception as e:
        raise ValueError(f"Error loading data: {str(e)}")

def _validate_date_gaps(df: pd.DataFrame, max_gap_days: int = 3) -> None:
    """Flag abnormal gaps in trading days (excluding weekends/holidays)."""
    gaps = df['Date'].diff().dt.days
    large_gaps = gaps[gaps > max_gap_days]
    if len(large_gaps) > 0:
        print(f"⚠️ Warning: {len(large_gaps)} gaps >{max_gap_days} days detected (e.g., {large_gaps.index[:3].tolist()})")
        print("   Note: Gaps may reflect weekends/holidays – validate against trading calendar.")

# Load data
df = load_brent_data()

ValueError: Error loading data: read_csv() got an unexpected keyword argument 'date_parser'

In [5]:
# 2. TREND ANALYSIS: Raw Price + Rolling Means
def plot_price_trend(df: pd.DataFrame, window_days: int = 90) -> None:
    """
    Visualize raw prices with multi-scale rolling means to isolate trend components.
    
    Args:
        window_days: Window size for primary rolling mean (default: 90 days = quarterly)
    """
    fig, ax = plt.subplots(figsize=(16, 7))
    
    # Raw price (light transparency)
    ax.plot(df['Date'], df['Price'], alpha=0.3, color='gray', label='Daily Price', linewidth=0.8)
    
    # Multi-scale rolling means
    for window, label, color in [(30, '30-day', 'blue'), (90, '90-day', 'red'), (365, '365-day', 'green')]:
        rolling_mean = df['Price'].rolling(window=window).mean()
        ax.plot(df['Date'], rolling_mean, label=f'{label} rolling mean', linewidth=2.5, color=color)
    
    # Highlight key volatility regimes with vertical spans
    regimes = [
        ('2008-07', '2009-06', 'Financial Crisis', '#ffe6e6'),
        ('2014-06', '2016-02', 'Oil Price Collapse', '#e6f7ff'),
        ('2020-03', '2020-04', 'Pandemic Shock', '#fff4e6'),
        ('2022-02', '2022-06', 'Russia-Ukraine War', '#f0e6ff')
    ]

    for start, end, label, color in regimes:
            ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.2, color=color, label=label if label else None)
        
    ax.set_title('Brent Oil Price Trend Analysis (1987–2022)', fontsize=16, fontweight='bold')
    ax.set_ylabel('Price (USD/barrel)', fontsize=13)
    ax.set_xlabel('Date', fontsize=13)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../docs/eda_trend_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_price_trend(df)

NameError: name 'df' is not defined

In [ ]:
# 3. STATIONARITY TESTING: Prices vs. Log Returns
def compute_log_returns(prices: pd.Series) -> pd.Series:
    """Calculate log returns for stationarity analysis."""
    return np.log(prices / prices.shift(1)).dropna()

def adf_test(series: pd.Series, label: str = "Series") -> dict:
    """
    Perform Augmented Dickey-Fuller test for stationarity.
    
    Returns:
        Dictionary with test statistic, p-value, and interpretation
    """
    result = adfuller(series.dropna(), autolag='AIC')
    return {
        'statistic': result[0],
        'p_value': result[1],
        'lags_used': result[2],
        'n_observations': result[3],
        'critical_values': result[4],
        'stationary': result[1] < 0.05,
        'interpretation': f"{label} is {'STATIONARY' if result[1] < 0.05 else 'NON-STATIONARY'} (p={result[1]:.4f})"
    }

# Compute log returns
log_returns = compute_log_returns(df['Price'])

# Run ADF tests
price_adf = adf_test(df['Price'], "Raw Prices")
returns_adf = adf_test(log_returns, "Log Returns")

# Display results
print("="*70)
print("STATIONARITY TESTING (Augmented Dickey-Fuller)")
print("="*70)
print(f"\n{'Raw Prices':<25} | Statistic: {price_adf['statistic']:>7.3f} | p-value: {price_adf['p_value']:>7.4f} | {price_adf['interpretation']}")
print(f"{'Log Returns':<25} | Statistic: {returns_adf['statistic']:>7.3f} | p-value: {returns_adf['p_value']:>7.4f} | {returns_adf['interpretation']}")
print("\n💡 Modeling Implication: Use LOG RETURNS for change point analysis (stationary series required)")
print("="*70)

# Visualize distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Raw prices distribution
sns.histplot(df['Price'], kde=True, ax=ax1, color='steelblue')
ax1.set_title('Distribution of Raw Prices', fontsize=14, fontweight='bold')
ax1.set_xlabel('Price (USD/barrel)')

# Log returns distribution
sns.histplot(log_returns, kde=True, ax=ax2, color='coral')
ax2.set_title('Distribution of Log Returns', fontsize=14, fontweight='bold')
ax2.set_xlabel('Log Return')

plt.tight_layout()
plt.savefig('../docs/eda_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 4. VOLATILITY ANALYSIS: Clustering and Event Alignment
def plot_volatility_clustering(df: pd.DataFrame, log_returns: pd.Series, window_days: int = 30) -> None:
    """
    Analyze volatility clustering using rolling standard deviation of log returns.
    
    Args:
        window_days: Rolling window for volatility calculation (default: 30 days)
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
    
    # Volatility (rolling std of log returns)
    volatility = log_returns.rolling(window=window_days).std()
    ax1.plot(df['Date'].iloc[window_days:], volatility.iloc[window_days:], 
             color='purple', linewidth=1.2)
    ax1.set_title(f'Volatility Clustering ({window_days}-day rolling std of log returns)', 
                  fontsize=15, fontweight='bold')
    ax1.set_ylabel('Volatility (σ)', fontsize=12)
    ax1.axhline(y=volatility.mean(), color='red', linestyle='--', alpha=0.7, label=f'Mean volatility: {volatility.mean():.4f}')
    ax1.legend()
    
    # Highlight extreme volatility periods (>2σ)
    high_vol_periods = volatility[volatility > volatility.mean() + 2*volatility.std()]
    if len(high_vol_periods) > 0:
        for date in high_vol_periods.index[::5]:  # Sample every 5th point to avoid overcrowding
            ax1.axvline(x=df.loc[date, 'Date'], color='orange', alpha=0.3, linewidth=0.8)
    # Price with volatility overlay (secondary y-axis)
    color = 'tab:blue'
    ax2.plot(df['Date'], df['Price'], color=color, alpha=0.7, linewidth=1)
    ax2.set_ylabel('Price (USD/barrel)', color=color, fontsize=12)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_title('Price Series with Volatility Overlay', fontsize=15, fontweight='bold')
    
    # Add volatility as shaded background
    ax2_vol = ax2.twinx()
    ax2_vol.fill_between(df['Date'].iloc[window_days:], 0, volatility.iloc[window_days:], 
                          alpha=0.3, color='purple', label='Volatility')
    ax2_vol.set_ylabel('Volatility', color='purple', fontsize=12)
    ax2_vol.tick_params(axis='y', labelcolor='purple')
    
    # Highlight key events from events.csv
    events = pd.read_csv('../data/events.csv', parse_dates=['date'])
    event_dates = events['date'].dt.to_pydatetime()
    for event_date in event_dates:
        if df['Date'].min() <= pd.Timestamp(event_date) <= df['Date'].max():
            ax2.axvline(x=event_date, color='red', linestyle='--', alpha=0.4, linewidth=1)
    
    ax2.legend(loc='upper left')
    ax2_vol.legend(loc='upper right')
    
    plt.tight_layout()
    plt.savefig('../docs/eda_volatility_clustering.png', dpi=300, bbox_inches='tight')
    plt.show()
    
plot_volatility_clustering(df, log_returns)


In [ ]:
# 5. AUTOCORRELATION ANALYSIS (Optional but informative for modeling)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# ACF for prices
plot_acf(df['Price'].dropna(), lags=30, ax=ax1, title='Autocorrelation: Raw Prices')
ax1.set_xlabel('Lag (days)')

# ACF for log returns
plot_acf(log_returns, lags=30, ax=ax2, title='Autocorrelation: Log Returns')
ax2.set_xlabel('Lag (days)')

plt.tight_layout()
plt.savefig('../docs/eda_autocorrelation.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 6. KEY FINDINGS & MODELING IMPLICATIONS
print("\n" + "="*70)
print("EDA KEY FINDINGS & MODELING IMPLICATIONS")
print("="*70)
print("\n1. TREND ANALYSIS:")
print("   → Strong non-linear trend with multiple structural breaks")
print("   → Multi-regime behavior evident (pre-2008, 2008–2014, 2014–2020, post-2020)")
print("   → Implication: Model log returns (detrended series) rather than raw prices")

print("\n2. STATIONARITY:")
print(f"   → Raw prices: NON-STATIONARY (ADF p-value = {price_adf['p_value']:.4f})")
print(f"   → Log returns: STATIONARY (ADF p-value = {returns_adf['p_value']:.4f})")
print("   → Implication: Change point model should be applied to log returns")

print("\n3. VOLATILITY CLUSTERING:")
print("   → Pronounced volatility clustering observed (ARCH effects)")
print("   → Extreme volatility periods align with major geopolitical events")
print("   → Implication: Consider σ₁ ≠ σ₂ in change point model (allow variance shift)")

print("\n4. EVENT ALIGNMENT:")
print("   → Volatility spikes consistently coincide with curated event dates")
print("   → Strong visual evidence of structural breaks around:")
print("      • Mar 2020 (pandemic + price war)")
print("      • Feb 2022 (Russia-Ukraine war)")
print("      • Sep 2019 (Saudi facility attack)")
print("   → Implication: Single-change-point model appropriate for focused event analysis")

print("\n5. MODELING RECOMMENDATION:")
print("   → Primary model: Bayesian change point on LOG RETURNS with mean shift (μ₁ → μ₂)")
print("   → Advanced extension: Allow variance shift (σ₁ → σ₂) for volatility regime changes")
print("   → Critical note: Temporal correlation ≠ causation – event alignment requires causal validation")
print("="*70)



In [ ]:

# 7. SAVE CLEANED DATASET FOR TASK 2
df_clean = df.copy()
df_clean['Log_Return'] = np.log(df_clean['Price'] / df_clean['Price'].shift(1))
df_clean.to_csv('../data/brent_clean.csv', index=False)
print("\n✓ Cleaned dataset saved to '../data/brent_clean.csv' (includes log returns)")